<a href="https://colab.research.google.com/github/rhozon/Banca-FAE/blob/master/Notebook_ForecastingScenariosRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:

# Colab Cell 1 — Instalação de Dependências
!pip install --quiet yahooquery prophet stable-baselines3 gymnasium tqdm pandas numpy plotly


In [38]:

# Colab Cell 2 — Histórico & Log-Retornos
from yahooquery import Ticker
import pandas as pd
import numpy as np
import gymnasium as gym
from tqdm.notebook import tqdm
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
import plotly.express as px

TICKERS = ["ZC=F","ZO=F","KE=F","GF=F","ZS=F","ZM=F","ZL=F"]
CUT     = pd.to_datetime("2025-03-22")

# 1) baixa 2 anos de preço ajustado
raw = (
    Ticker(TICKERS)
    .history(period="2y", interval="1d")["adjclose"]
    .dropna()
)
# 2) normaliza índice e coluna date
df2 = raw.reset_index()
df2["date"] = pd.to_datetime(df2["date"].astype(str).str[:10])
# 3) pivot para wide e drop NAs
df_prices = (
    df2
    .pivot(index="date", columns="symbol", values="adjclose")
    .sort_index()
    .dropna(how="any")
)
# 4) calcula log-retornos em formato long
log_ret = np.log(df_prices).diff().dropna()
df_hist = (
    log_ret
    .reset_index()
    .melt(id_vars="date", var_name="asset", value_name="logret")
)


/usr/local/lib/python3.11/dist-packages/yahooquery/utils/__init__.py:1470: FutureWarning:

'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.

/usr/local/lib/python3.11/dist-packages/yahooquery/utils/__init__.py:1470: FutureWarning:

'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.

/usr/local/lib/python3.11/dist-packages/yahooquery/utils/__init__.py:1470: FutureWarning:

'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.

/usr/local/lib/python3.11/dist-packages/yahooquery/utils/__init__.py:1470: FutureWarning:

'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.

/usr/local/lib/python3.11/dist-packages/yahooquery/utils/__init__.py:1470: FutureWarning:

'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.

/usr/local/lib/python3.11/dist-packages/yahooquery/utils/__init__.py:1470: FutureWarning:


In [39]:

# Colab Cell 3 — Forecast com Prophet (3 cenários)
from prophet import Prophet

candidate_cps = ["2020-01-01","2022-01-01","2024-01-01"]
HORIZON, N_SCEN = 30, 3

sims = []
for asset, grp in df_hist.groupby("asset"):
    ds_min, ds_max = grp["date"].min(), grp["date"].max()
    cps = [pd.to_datetime(cp) for cp in candidate_cps if ds_min <= pd.to_datetime(cp) <= ds_max]

    m = Prophet(
        changepoints=cps or None,
        interval_width=0.95,
        uncertainty_samples=N_SCEN,
        daily_seasonality=False
    )
    dfp = grp.rename(columns={"date":"ds","logret":"y"})[["ds","y"]]
    m.fit(dfp)

    future = m.make_future_dataframe(periods=HORIZON, freq="D")
    ps     = m.predictive_samples(future)
    yhat   = ps["yhat"]

    # detecta orientação
    if yhat.shape == (N_SCEN, len(future)):
        get_sim = lambda r: yhat[r]
    elif yhat.shape == (len(future), N_SCEN):
        get_sim = lambda r: yhat[:, r]
    else:
        raise ValueError(f"Forma inesperada de yhat: {yhat.shape}")

    dates = future["ds"].values
    L     = len(dates)
    for rep in range(N_SCEN):
        sims.append(pd.DataFrame({
            "asset": [asset]*L,
            "date":  dates,
            ".rep":  [str(rep+1)]*L,
            ".sim":  get_sim(rep)
        }))
sims_all = pd.concat(sims, ignore_index=True)


DEBUG:cmdstanpy:input tempfile: /tmp/tmp_qovajij/rnalg_zd.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_qovajij/gij648dh.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=8647', 'data', 'file=/tmp/tmp_qovajij/rnalg_zd.json', 'init=/tmp/tmp_qovajij/gij648dh.json', 'output', 'file=/tmp/tmp_qovajij/prophet_modeli70b7kg5/prophet_model-20250502194209.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
19:42:09 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:42:09 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_qovajij/0ubh6vzx.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_qovajij/2vlv0zkj.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/l

In [40]:


# Colab Cell 4 — Plot interativo com Plotly
import plotly.express as px

CUT = pd.to_datetime("2025-03-22")

# prepara df_plot2 (histórico + forecasts)
hist2 = df_hist[df_hist["date"]<=CUT].copy()
hist2["rep"] = "Histórico"
hist2 = hist2.rename(columns={"logret":"logret"})

f2 = (
    sims_all[sims_all["date"]>CUT]
    .rename(columns={".rep":"rep", ".sim":"logret"})
)

df_plot2 = pd.concat([
    hist2[["asset","date","rep","logret"]],
    f2[["asset","date","rep","logret"]]
], ignore_index=True)

fig = px.line(
    df_plot2,
    x="date",
    y="logret",
    color="rep",
    line_dash="rep",
    facet_col="asset",
    facet_col_wrap=2,
    labels={"date":"Data","logret":"Log-retorno","rep":"Série"},
    title="Histórico até 2025-03-22 e 3 Cenários de Forecast"
)

# adiciona linha de cutoff
fig.add_shape(
    type="line",
    x0=CUT, x1=CUT,
    y0=0, y1=1,
    xref="x", yref="paper",
    line=dict(color="red", dash="dot")
)
fig.add_annotation(
    x=CUT, y=1.02,
    xref="x", yref="paper",
    text="Cutoff",
    showarrow=False,
    font=dict(color="red")
)

fig.update_layout(
    legend_title_text="Série",
    width=1000, height=600,
    margin=dict(l=40,r=200,t=80,b=40)
)
fig.update_xaxes(matches=None, nticks=5)
fig.show()


In [41]:

# Colab Cell 4 — Classifica Cenários e Executa RL Dia-a-Dia (RAW_ENV)
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from tqdm.notebook import tqdm
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
import gymnasium as gym

WINDOW_SIZE     = 10
TOTAL_TIMESTEPS = 10_000
SEED            = 42

# Ambiente sobre log-retornos
class ReturnEnv(gym.Env):
    metadata = {"render.modes": ["human"]}
    def __init__(self, ret_series, window_size):
        super().__init__()
        self.returns     = ret_series.reset_index(drop=True).astype(np.float32)
        self.window_size = window_size
        # observação = janela de retornos
        self.observation_space = gym.spaces.Box(
            low=-np.inf, high=np.inf,
            shape=(window_size,), dtype=np.float32
        )
        self.action_space = gym.spaces.Discrete(3)

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        self.t = self.window_size
        obs = self.returns[self.t-self.window_size : self.t].values
        return obs, {}  # Gymnasium: (obs, info)

    def step(self, action):
        reward     = float(self.returns[self.t] * (1 if action==2 else 0))
        self.t    += 1
        terminated = self.t >= len(self.returns)
        obs        = (self.returns[self.t-self.window_size : self.t].values
                      if not terminated else np.zeros(self.window_size, dtype=np.float32))
        return obs, reward, terminated, False, {}  # (obs, reward, term., trunc., info)

# Extrai a série de log-retornos simulados
def get_series(asset, rep):
    return (
        sims_all
        .loc[(sims_all["asset"]==asset)&(sims_all[".rep"]==rep), ".sim"]
        .reset_index(drop=True)
    )

action_map = {0:"SELL",1:"HOLD",2:"BUY"}
records    = []

for asset in tqdm(sims_all["asset"].unique(), desc="Assets"):
    for rep in sims_all[".rep"].unique():
        series = get_series(asset, rep)

        # 1) treina com DummyVecEnv
        def make_env(): return ReturnEnv(series, WINDOW_SIZE)
        train_env = DummyVecEnv([make_env])
        model     = DQN("MlpPolicy", train_env, seed=SEED, verbose=0)
        model.learn(total_timesteps=TOTAL_TIMESTEPS)

        # 2) executa passo-a-passo no raw_env
        raw_env = make_env()
        obs, _  = raw_env.reset()
        for i in range(len(series)-WINDOW_SIZE):
            action, _ = model.predict(obs, deterministic=True)
            date = sims_all.loc[
                (sims_all["asset"]==asset)&(sims_all[".rep"]==rep),
                "date"
            ].iloc[WINDOW_SIZE + i]

            records.append({
                "asset":  asset,
                "rep":    rep,
                "date":   pd.to_datetime(date),
                "logret": series.iloc[WINDOW_SIZE + i],
                "action": action_map[int(action)]
            })

            obs, _, terminated, _, _ = raw_env.step(action)
            if terminated:
                break

df_recos_full = pd.DataFrame(records)
display(df_recos_full.tail())


Assets:   0%|          | 0/7 [00:00<?, ?it/s]

,asset,rep,date,logret,action
10978,ZS=F,3,2025-05-28,0.006751,HOLD
10979,ZS=F,3,2025-05-29,0.012197,HOLD
10980,ZS=F,3,2025-05-30,-0.008748,BUY
10981,ZS=F,3,2025-05-31,0.000793,BUY
10982,ZS=F,3,2025-06-01,0.016867,BUY


In [45]:

# Colab Cell X — Tabela interativa de recomendações OOS (corrigido)

import numpy as np
import pandas as pd
from google.colab import data_table
import ipywidgets as widgets
from IPython.display import display

# habilita DataTable interativo no Colab
data_table.enable_dataframe_formatter()

# cria coluna “oos” (fora da amostra) — pressupõe df_recos_full e CUT já existentes
df_recos_full["oos"] = df_recos_full["date"] > CUT

# função de estilo para colorir a coluna de ações
def color_action(val):
    if val == "BUY":
        return "background-color: #c6efce; color: #006100"
    if val == "SELL":
        return "background-color: #ffc7ce; color: #9c0006"
    if val == "HOLD":
        return "background-color: #ffeb9c; color: #9c6500"
    return ""

# dropdowns para filtrar
w_ticker = widgets.Dropdown(
    options=sorted(df_recos_full["asset"].unique()),
    description="Ticker:"
)
w_rep = widgets.Dropdown(
    options=sorted(df_recos_full["rep"].unique()),
    description="Cenário:"
)

# função que exibe a tabela filtrada e estilizada
def mostrar_recs(asset, rep):
    df = (
        df_recos_full
        .query("asset == @asset and rep == @rep and oos")
        .sort_values("date")
        .reset_index(drop=True)
    )
    styled = (
        df.style
          .applymap(color_action, subset=["action"])
          .set_caption(f"Recomendações OOS — {asset} Cenário {rep}")
    )
    display(styled)

# widget interativo
display(widgets.VBox([
    widgets.HTML("<b>Selecione Ticker e Cenário para ver Recomendações Fora da Amostra</b>"),
    widgets.HBox([w_ticker, w_rep])
]))
widgets.interact(mostrar_recs, asset=w_ticker, rep=w_rep)

# resumo estatístico das ações OOS
resumo = (
    df_recos_full[df_recos_full["oos"]]
    .groupby(["asset","rep","action"])
    .size()
    .unstack(fill_value=0)
    .sort_index()
)
display(resumo.style.set_caption("Contagem de BUY / HOLD / SELL por Ticker e Cenário"))


interactive(children=(Dropdown(description='Ticker:', options=('GF=F', 'KE=F', 'ZC=F', 'ZL=F', 'ZM=F', 'ZO=F',…